In [117]:
cd/Users/paavankumars/Desktop/ML/

/Users/paavankumars/Desktop/ML


In [118]:
ls 

1_6_ML.ipynb                  ml.ipynb
25_may_ml.ipynb               ml4.ipynb
Iris (3).csv                  music.csv
ML_2.ipynb.webloc             music.ipynb
ML_Housingprice.ipynb.webloc  pet_classifier.ipynb
ML_lab3.ipynb.webloc          play.csv
Untitled.ipynb                text_classifier.ipynb
X.pkl                         training.ipynb
email.csv                     wine-reviews.csv
emailspam_ml.ipynb            y.pkl
heart.csv


In [119]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_hub as hub

In [120]:
df = pd.read_csv("wine-reviews.csv", usecols=['country', 'description', 'points', 'price', 'variety', 'winery'])

In [121]:
df.head()

,country,description,points,price,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Pinot Noir,Sweet Cheeks


In [122]:
df = df.dropna(subset=("description", "points"))

In [123]:
df.head()

,country,description,points,price,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Pinot Noir,Sweet Cheeks


In [124]:
df["label"] = (df.points >= 90).astype(int)
df = df[["description", "label"]]

In [125]:
df.tail()

,description,label
129966,Notes of honeysuckle and cantaloupe sweeten th...,1
129967,Citation is given as much as a decade of bottl...,1
129968,Well-drained gravel soil gives this wine its c...,1
129969,"A dry style of Pinot Gris, this is crisp with ...",1
129970,"Big, rich and off-dry, this is powered by inte...",1


In [126]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(data)), int(0.9*len(data))])

In [127]:
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
    df = dataframe.copy()
    labels = df.pop('label')
    df = df["description"]
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [128]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

In [129]:
encoder = tf.keras.layers.TextVectorization(max_tokens=2000)
encoder.adapt(train_data.map(lambda text, label: text))

In [130]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'and', 'the', 'a', 'of', 'with', 'this', 'is', 'wine',
       'flavors', 'in', 'it', 'to', 'its', 'on', 'fruit', 'aromas',
       'palate', 'that'], dtype='<U17')

In [131]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        mask_zero=True
    ),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [137]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [138]:
model.evaluate(train_data)
model.evaluate(valid_data)

13/13 [==============================] - 2s 124ms/step - loss: 0.3507 - accuracy: 0.8387


[0.35072043538093567, 0.8386550545692444]

In [139]:
history = model.fit(train_data, epochs=5, validation_data=valid_data)

Epoch 1/5
102/102 [==============================] - 58s 537ms/step - loss: 0.3367 - accuracy: 0.8486 - val_loss: 0.3511 - val_accuracy: 0.8377
Epoch 2/5
102/102 [==============================] - 40s 386ms/step - loss: 0.3314 - accuracy: 0.8511 - val_loss: 0.3557 - val_accuracy: 0.8377
Epoch 3/5
102/102 [==============================] - 29s 283ms/step - loss: 0.3289 - accuracy: 0.8517 - val_loss: 0.3572 - val_accuracy: 0.8345
Epoch 4/5
102/102 [==============================] - 28s 274ms/step - loss: 0.3271 - accuracy: 0.8532 - val_loss: 0.3512 - val_accuracy: 0.8376
Epoch 5/5
102/102 [==============================] - 26s 256ms/step - loss: 0.3254 - accuracy: 0.8530 - val_loss: 0.3544 - val_accuracy: 0.8334


In [140]:
model.evaluate(test_data)

13/13 [==============================] - 1s 93ms/step - loss: 0.3475 - accuracy: 0.8432


[0.3475407660007477, 0.8432066440582275]